In [10]:
from keras_radam import RAdam

In [11]:
import tensorflow as tf
import tensorflow.keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
import cv2

In [12]:
# img_dir = '/Users/User/303/KT_32px/food_competition_KT_set1/train/'
# categoris = os.listdir(img_dir)
# nb_categoris = len(categoris)

In [13]:
# data = []
# labels = []
# for i in range(nb_categoris) :
#     a = glob.glob(img_dir+'/'+categoris[i]+'/*.jpg')
#     for j in a :
#         image=tensorflow.keras.preprocessing.image.load_img(j, color_mode='rgb')
#         image=np.array(image)
#         data.append(image)
#         labels.append(i)

In [14]:
# data=np.array(data)
# data=data/255.0
# data.shape

In [15]:
data=np.load('data.npy')
#data=data/255.0
#data[0]
#data.shape

In [16]:
labels=np.load('labels.npy')
labels.shape

(10000, 50)

In [17]:
from sklearn.model_selection import KFold
np.random.seed(0)
k=5
kf=KFold(n_splits=k,shuffle=True,random_state = 5)

In [20]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D,ReLU,LeakyReLU,ELU,BatchNormalization,Dropout,GlobalAveragePooling2D,Input,AveragePooling2D, Activation
from tensorflow.keras.layers import MaxPool2D,Concatenate
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam,SGD,Nadam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.regularizers import l2
from sklearn.model_selection import KFold
import numpy as np


In [21]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import MaxPool2D
from keras.layers import BatchNormalization, Dropout

#act = tf.nn.leaky_relu
act = tf.nn.relu

np.random.seed(0)

#65퍼 Best batch32 epoch 200 adam()
def model_fn():
  with tf.device("/gpu:0"):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation=act, kernel_initializer='he_normal', padding='same', input_shape=(128, 128, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Conv2D(256, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation=act, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='softmax'))
    return model

In [22]:
model_fn().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                        

In [23]:
# def lr_schedule(epoch):
#     lr = 1e-3
#     if epoch > 80:
#         lr *= 0.5e-3
#     elif epoch > 60:
#         lr *= 1e-3
#     elif epoch > 40:
#         lr *= 1e-2
#     elif epoch > 2:
#         lr *= 1e-1
#     print('Learning rate: ', lr)
#     return lr

In [24]:
# lr_scheduler = LearningRateScheduler(lr_schedule)

# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)

# callbacks = [lr_reducer, lr_scheduler]


In [25]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
pat = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

fold_no = 1
BATCH_SIZE = 128
EPOCHS = 100
acc_per_fold = []
loss_per_fold = []

np.random.seed(3)

for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    trainGenerator = ImageDataGenerator(rescale=1/255., horizontal_flip=True,
                        rotation_range = 15, shear_range=0.2)
    trainiterator = trainGenerator.flow(X_train, y_train, batch_size=BATCH_SIZE)

    testGenerator = ImageDataGenerator(rescale=1/255., horizontal_flip=True,
                        rotation_range = 15, shear_range=0.2)
    testiterator = testGenerator.flow(X_test, y_test, batch_size=BATCH_SIZE)


    model = model_fn()
    
    model.compile(loss='categorical_crossentropy', optimizer=RAdam(), metrics=['accuracy'])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit_generator(trainiterator,
                  epochs=EPOCHS,
                  validation_data= testiterator, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate_generator(testiterator, verbose=1)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...


C:\Users\User\AppData\Local\Temp\ipykernel_12440\1645387615.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(trainiterator,


Epoch 1/100
63/63 [==============================] - 30s 353ms/step - loss: 4.2410 - accuracy: 0.0599 - val_loss: 4.1252 - val_accuracy: 0.0350
Epoch 2/100
63/63 [==============================] - 21s 333ms/step - loss: 3.1511 - accuracy: 0.1857 - val_loss: 4.4665 - val_accuracy: 0.0520
Epoch 3/100
63/63 [==============================] - 21s 330ms/step - loss: 2.6317 - accuracy: 0.2816 - val_loss: 4.5504 - val_accuracy: 0.0585
Epoch 4/100
63/63 [==============================] - 21s 337ms/step - loss: 2.3273 - accuracy: 0.3470 - val_loss: 4.8075 - val_accuracy: 0.0740
Epoch 5/100
63/63 [==============================] - 21s 330ms/step - loss: 2.1088 - accuracy: 0.4016 - val_loss: 4.5843 - val_accuracy: 0.1145
Epoch 6/100
63/63 [==============================] - 21s 333ms/step - loss: 1.9527 - accuracy: 0.4461 - val_loss: 3.6584 - val_accuracy: 0.1715
Epoch 7/100
63/63 [==============================] - 21s 330ms/step - loss: 1.8225 - accuracy: 0.4749 - val_loss: 3.1612 - val_accuracy:

63/63 [==============================] - 21s 333ms/step - loss: 0.2227 - accuracy: 0.9294 - val_loss: 1.3177 - val_accuracy: 0.6910
Epoch 58/100
63/63 [==============================] - 21s 340ms/step - loss: 0.2234 - accuracy: 0.9265 - val_loss: 1.3834 - val_accuracy: 0.6745
Epoch 59/100
63/63 [==============================] - 21s 332ms/step - loss: 0.2184 - accuracy: 0.9285 - val_loss: 1.2868 - val_accuracy: 0.6990
Epoch 60/100
63/63 [==============================] - 21s 332ms/step - loss: 0.2122 - accuracy: 0.9316 - val_loss: 1.2118 - val_accuracy: 0.7080
Epoch 61/100
63/63 [==============================] - 21s 338ms/step - loss: 0.1901 - accuracy: 0.9390 - val_loss: 1.4143 - val_accuracy: 0.6900
Epoch 62/100
63/63 [==============================] - 21s 332ms/step - loss: 0.1939 - accuracy: 0.9377 - val_loss: 1.3928 - val_accuracy: 0.6885
Epoch 63/100
63/63 [==============================] - 21s 337ms/step - loss: 0.1991 - accuracy: 0.9352 - val_loss: 1.5131 - val_accuracy: 0.682

C:\Users\User\AppData\Local\Temp\ipykernel_12440\1645387615.py:42: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(testiterator, verbose=1)


16/16 [==============================] - 4s 258ms/step - loss: 1.4273 - accuracy: 0.7120
Score for fold 1: loss of 1.4273148775100708; accuracy of 71.20000123977661%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/100
63/63 [==============================] - 26s 339ms/step - loss: 4.2046 - accuracy: 0.0615 - val_loss: 4.3424 - val_accuracy: 0.0340
Epoch 2/100
63/63 [==============================] - 21s 335ms/step - loss: 3.1439 - accuracy: 0.1791 - val_loss: 5.7864 - val_accuracy: 0.0335
Epoch 3/100
63/63 [==============================] - 21s 332ms/step - loss: 2.6494 - accuracy: 0.2690 - val_loss: 5.6554 - val_accuracy: 0.0490
Epoch 4/100
63/63 [==============================] - 21s 332ms/step - loss: 2.3231 - accuracy: 0.3491 - val_loss: 4.8182 - val_accuracy: 0.0720
Epoch 5/100
63/63 [==============================] - 21s 338ms/step - loss: 2.0858 - accuracy: 0.4042 - val_loss: 4.5818 - val_accuracy: 0.0750
Epoch 6/100
63/63

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



63/63 [==============================] - 21s 336ms/step - loss: 0.4483 - accuracy: 0.8571 - val_loss: 1.3638 - val_accuracy: 0.6405
Epoch 38/100
63/63 [==============================] - 21s 341ms/step - loss: 0.4373 - accuracy: 0.8596 - val_loss: 1.2307 - val_accuracy: 0.6780
Epoch 39/100
63/63 [==============================] - 21s 334ms/step - loss: 0.4146 - accuracy: 0.8686 - val_loss: 1.1726 - val_accuracy: 0.6800
Epoch 40/100
63/63 [==============================] - 21s 337ms/step - loss: 0.3893 - accuracy: 0.8725 - val_loss: 1.4314 - val_accuracy: 0.6365
Epoch 41/100
63/63 [==============================] - 21s 334ms/step - loss: 0.3855 - accuracy: 0.8765 - val_loss: 1.3533 - val_accuracy: 0.6640
Epoch 42/100
62/63 [============================>.] - ETA: 0s - loss: 0.3725 - accuracy: 0.8792

KeyboardInterrupt: 

In [ ]:
accuracy_history

In [ ]:
# 복잡한 모델 epoch 50
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:
np.mean(accuracy_history)

In [ ]:
accuracy_history